In [185]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [186]:
df = pd.read_csv("../Resources/combined_cleaned.csv")
df.head()

,id,gender,customer_type,age,type_of_travel,travel_class,flight_distance,inflight_wifi_service,departure/arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,on-board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,4,3,4,3,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,1,5,4,5,4,4,4,4,3,4,5,0,0.0,satisfied
2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,4,2,2,2,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,2,3,4,4,1,1,1,1,3,1,4,0,6.0,satisfied
4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,3,4,1,2,2,2,2,2,4,2,4,0,20.0,satisfied


In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 129487 non-null  int64  
 1   gender                             129487 non-null  object 
 2   customer_type                      129487 non-null  object 
 3   age                                129487 non-null  int64  
 4   type_of_travel                     129487 non-null  object 
 5   travel_class                       129487 non-null  object 
 6   flight_distance                    129487 non-null  int64  
 7   inflight_wifi_service              129487 non-null  int64  
 8   departure/arrival_time_convenient  129487 non-null  int64  
 9   ease_of_online_booking             129487 non-null  int64  
 10  gate_location                      129487 non-null  int64  
 11  food_and_drink                     1294

In [188]:
df["arrival_delay_in_minutes"] = df["arrival_delay_in_minutes"].astype('int')

In [189]:
# Imputing

df.drop(columns=["id"], inplace=True)

## Preprocessing Pipeline

In [190]:
# # Dropping columns

# class DropColumn(BaseEstimator, TransformerMixin):
#     def __init__(self, column):
#         self.column = column
        
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         # Ensure that we return a DataFrame after dropping the column
#         return X.drop(columns=[self.column]).reset_index(drop=True)

In [191]:
# Binning for columns with too many categories

# Define the custom transformer for multiple columns
class BinningTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, bins_dict, labels_dict):
        self.bins_dict = bins_dict
        self.labels_dict = labels_dict

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_binned = X.copy()
        for column in self.bins_dict.keys():
            X_binned[column] = pd.cut(X_binned[column], bins=self.bins_dict[column], labels=self.labels_dict[column], include_lowest=True)
        return X_binned
    # def transform(self, X):
    #     # Create bins for each feature based on the provided bins_dict
    #     binned_data = pd.DataFrame()
    #     for feature in X.columns:
    #         binned_data[feature] = pd.cut(X[feature], bins=self.bins_dict[feature], labels=self.labels_dict[feature])
    #     return binned_data

    def get_feature_names_out(self, input_features=None):
        # Generate feature names based on the labels_dict
        if input_features is None:
            return self.labels_dict.values()
        else:
            return [f"{feature}_{label}" for feature in input_features for label in self.labels_dict[feature]]

# Define bins and labels for each column
bins_dict = {
    'flight_distance': [0, 99, 199, 299, 399, 499, 599, 799, 999, 1499, 1999, 2999, 4999],
    'departure_delay_in_minutes': [0, 30, 60, 120, 180, 240, 300, 360, 420, 480, 2000],
    'arrival_delay_in_minutes': [0, 30, 60, 120, 180, 240, 300, 360, 420, 480, 2000]
}

labels_dict = {
    'flight_distance': ["0-100", "100-199", "200-299", "300-399", "400-499", "500-599", "600-799", "800-999", "1000-1499", "1500-1999", "2000-2999", "3000-4999"],
    'departure_delay_in_minutes':["0-29", "30-59", "60-119", "120-179", "180-239", "240-299", "300-359", "360-419", "420-479", "480-1999"],
    'arrival_delay_in_minutes': ["0-29", "30-59", "60-119", "120-179", "180-239", "240-299", "300-359", "360-419", "420-479", "480-1999"]
}

In [193]:
binning_transformer = BinningTransformer(bins_dict, labels_dict)
binned_df = binning_transformer.fit_transform(df)
binned_df.head()

,gender,customer_type,age,type_of_travel,travel_class,flight_distance,inflight_wifi_service,departure/arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,on-board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,Female,Loyal Customer,52,Business travel,Eco,100-199,5,4,3,4,3,4,3,5,5,5,5,2,5,5,30-59,30-59,satisfied
1,Female,Loyal Customer,36,Business travel,Business,2000-2999,1,1,3,1,5,4,5,4,4,4,4,3,4,5,0-29,0-29,satisfied
2,Male,disloyal Customer,20,Business travel,Eco,100-199,2,0,2,4,2,2,2,2,4,1,3,2,2,2,0-29,0-29,neutral or dissatisfied
3,Male,Loyal Customer,44,Business travel,Business,3000-4999,0,0,0,2,3,4,4,1,1,1,1,3,1,4,0-29,0-29,satisfied
4,Female,Loyal Customer,49,Business travel,Eco,1000-1499,2,3,4,3,4,1,2,2,2,2,2,4,2,4,0-29,0-29,satisfied


In [194]:
# Label Encoding for binary features

class OrdinalEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = OrdinalEncoder()

    def fit(self, X, y=None):
        self.encoder.fit(X)
        return self

    def transform(self, X):
        # Transform the data and convert it back to a DataFrame
        X_encoded = self.encoder.transform(X)
        return pd.DataFrame(X_encoded, columns=self.encoder.get_feature_names_out(input_features=X.columns))
        
binary_features = ['gender', 'customer_type', 'type_of_travel', 'satisfaction']
binary_transformer = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoderTransformer())
])


In [195]:
# Scaling for numeric features
numeric_features = ['age', 'inflight_wifi_service',
       'departure/arrival_time_convenient', 'ease_of_online_booking',
       'gate_location', 'food_and_drink', 'online_boarding', 'seat_comfort',
       'inflight_entertainment', 'on-board_service', 'leg_room_service',
       'baggage_handling', 'checkin_service', 'inflight_service',
       'cleanliness']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [196]:
# One-Hot Encoding for categorical features 
categorical_features = ['travel_class', 'flight_distance', 'departure_delay_in_minutes', 'arrival_delay_in_minutes']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) 
    # Set sparse_output=False to return a dense array
    ])

In [197]:
preprocessor = ColumnTransformer(
    transformers=[
        # ('drop', DropColumn(column='id')),
        # ('binning', BinningTransformer(bins_dict=bins_dict, labels_dict=labels_dict), 
        #      ['flight_distance', 'departure_delay_in_minutes', 'arrival_delay_in_minutes']),
        ('binary', binary_transformer, binary_features),
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ], remainder='passthrough') # Keeps the other columns

In [198]:
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit and transform DataFrame 
df_transformed = pipeline.fit_transform(df)

# encoded_feature_names = (numeric_features + binary_features +
#   list(preprocessor.transformers_[2][1]['onehot'].get_feature_names_out(categorical_features)))

df_pipe = pd.DataFrame(df_transformed)
print(df_pipe.shape)

# feature_names = []

# # Get feature names from the binning transformer if applicable
# if hasattr(pipeline.named_steps['preprocessor'].named_transformers_['binning'], 'get_feature_names_out'):
#     binned_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['binning'].get_feature_names_out()
#     feature_names.extend(binned_feature_names)
# else:
#     # If it doesn't provide feature names, you can manually define them based on your bins
#     for feature in ['flight_distance', 'departure_delay_in_minutes', 'arrival_delay_in_minutes']:
#         feature_names.extend([f"{feature}_{label}" for label in labels_dict[feature]])
# print("\n".join(feature_names))

# # Get feature names from the numeric transformer
# if hasattr(pipeline.named_steps['preprocessor'].named_transformers_['numeric'], 'get_feature_names_out'):
#     numeric_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['numeric'].get_feature_names_out()
#     feature_names.extend(numeric_feature_names)

# # Get feature names from the categorical transformer
# if hasattr(pipeline.named_steps['preprocessor'].named_transformers_['categorical'], 'get_feature_names_out'):
#     categorical_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['categorical'].get_feature_names_out()
#     feature_names.extend(categorical_feature_names)
# #     print(len(feature_names))
# #     print("\n".join(feature_names))

# df_pipe.columns = feature_names
df_pipe.head()

(129487, 4779)


0     1     2     3         4         5         6         7         8     \
0   0.0   0.0   0.0   1.0  0.831567  1.708851  0.617410  0.173519  0.800227   
1   0.0   0.0   0.0   1.0 -0.226807 -1.300410 -1.347508  0.173519 -1.546271   
2   1.0   1.0   0.0   0.0 -1.285180 -0.548095 -2.002480 -0.539923  0.800227   
3   1.0   0.0   0.0   1.0  0.302380 -2.052726 -2.002480 -1.966806 -0.764105   
4   0.0   0.0   0.0   1.0  0.633122 -0.548095 -0.037562  0.886960  0.018061   

       9         10        11        12        13        14        15    \
0 -0.153910  0.553276 -0.334749  1.230701  1.256225  1.252860  1.159342   
1  1.349963  0.553276  1.181364  0.481157  0.479240  0.493055  0.311941   
2 -0.905846 -0.927497 -1.092806 -1.017931  0.479240 -1.786361 -0.535461   
3 -0.153910  0.553276  0.423307 -1.767475 -1.851712 -1.786361 -2.230265   
4  0.598027 -1.667884 -1.092806 -1.017931 -1.074728 -1.026556 -1.382863   

       16        17        18    19    20    21    22    23    24    25    \
0 -1.031669  1.153847  1.304624   0.0   1.0   0.0   0.0   0.0   0.0   0.0   
1 -0.241868  0.303947  1.304624   1.0   0.0   0.0   0.0   0.0   0.0   0.0   
2 -1.031669 -1.395853 -0.979144   0.0   1.0   0.0   0.0   0.0   0.0   0.0   
3 -0.241868 -2.245753  0.543368   1.0   0.0   0.0   0.0   0.0   0.0   0.0   
4  0.547933 -1.395853  0.543368   0.0   1.0   0.0   0.0   0.0   0.0   0.0   

   26    27    28    29    30    31    32    33    34    35    36    37    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   38    39    40    41    42    43    44    45    46    47    48    49    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   50    51    52    53    54    55    56    57    58    59    60    61    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   62    63    64    65    66    67    68    69    70    71    72    73    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   74    75    76    77    78    79    80    81    82    83    84    85    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   86    87    88    89    90    91    92    93    94    95    96    97    \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 

In [98]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 26 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   gender                             129487 non-null  object  
 1   customer_type                      129487 non-null  object  
 2   age                                129487 non-null  int64   
 3   type_of_travel                     129487 non-null  object  
 4   travel_class                       129487 non-null  object  
 5   flight_distance                    129487 non-null  int64   
 6   inflight_wifi_service              129487 non-null  int64   
 7   departure/arrival_time_convenient  129487 non-null  int64   
 8   ease_of_online_booking             129487 non-null  int64   
 9   gate_location                      129487 non-null  int64   
 10  food_and_drink                     129487 non-null  int64   
 11  online_boarding           

Index(['gender', 'customer_type', 'age', 'type_of_travel', 'travel_class',
       'flight_distance', 'inflight_wifi_service',
       'departure/arrival_time_convenient', 'ease_of_online_booking',
       'gate_location', 'food_and_drink', 'online_boarding', 'seat_comfort',
       'inflight_entertainment', 'on-board_service', 'leg_room_service',
       'baggage_handling', 'checkin_service', 'inflight_service',
       'cleanliness', 'departure_delay_in_minutes', 'arrival_delay_in_minutes',
       'satisfaction', 'flight_distance_binned',
       'departure_delay_in_minutes_binned', 'arrival_delay_in_minutes_binned'],
      dtype='object')

In [85]:
# Split preprocessed data into features and target arrays
X = df.drop(columns=["satisfaction"])
y = df.satisfaction
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 26 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   id                                 129487 non-null  int64   
 1   gender                             129487 non-null  object  
 2   customer_type                      129487 non-null  object  
 3   age                                129487 non-null  int64   
 4   type_of_travel                     129487 non-null  object  
 5   travel_class                       129487 non-null  object  
 6   flight_distance                    129487 non-null  int64   
 7   inflight_wifi_service              129487 non-null  int64   
 8   departure/arrival_time_convenient  129487 non-null  int64   
 9   ease_of_online_booking             129487 non-null  int64   
 10  gate_location                      129487 non-null  int64   
 11  food_and_drink            

In [174]:
# # Correlation analysis

# Get the feature names after one-hot encoding
encoded_feature_names = (numeric_features + binary_features +
  list(preprocessor.transformers_[2][1]['onehot'].get_feature_names_out(categorical_features)))
df_pipe = pd.DataFrame(df_pipe, columns = encoded_feature_names)
df_pipe.head()

KeyError: 'onehot'